## Step 1: Import Libraries

In [1]:
%run ../make_clean_names.py

In [2]:
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import logging

import polars as pl
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def create_session():
    session = requests.Session()
    retries = Retry(
        total=3,
        backoff_factor=1,
        status_forcelist=[429, 500, 502, 503, 504]
    )
    session.mount('https://', HTTPAdapter(max_retries=retries, pool_maxsize=10))
    return session

# Get API key from environment variables
FMP_API_KEY = os.getenv('FMP_API_KEY')
if not FMP_API_KEY:
    raise ValueError("FMP_API_KEY not found in environment variables")

## Step 2: Import Symbols

In [3]:
def load_symbols(file_path: str) -> List[str]:
    """Load symbols from a text file"""
    try:
        with open(file_path, 'r') as f:
            symbols = [line.strip() for line in f if line.strip()]
        print(f"Loaded {len(symbols)} symbols from {file_path}")
        return symbols
    except Exception as e:
        print(f"Error loading symbols: {str(e)}")
        return []

symbols_file = '../tickers.txt'
symbols = load_symbols(symbols_file)

if symbols:
    print("Symbols:", symbols)
else:
    print("No symbols loaded.")

Loaded 61 symbols from ../tickers.txt
Symbols: ['AAPL', 'MSFT', 'GOOGL', 'GOOG', 'META', 'NVDA', 'AVGO', 'ORCL', 'CRM', 'ACN', 'ADBE', 'CSCO', 'INTC', 'NFLX', 'DIS', 'CMCSA', 'VZ', 'T', 'AMZN', 'TSLA', 'HD', 'MCD', 'NKE', 'SBUX', 'TGT', 'LOW', 'WMT', 'PG', 'KO', 'PEP', 'COST', 'BRK-B', 'JPM', 'BAC', 'WFC', 'GS', 'MS', 'BLK', 'UNH', 'JNJ', 'PFE', 'ABBV', 'MRK', 'LLY', 'CAT', 'BA', 'HON', 'UPS', 'RTX', 'GE', 'XOM', 'CVX', 'COP', 'SLB', 'LIN', 'APD', 'ECL', 'PLD', 'AMT', 'CCI', 'OSW']


## Step 3: Extract Data from FMP into Polars

In [4]:
def fetch_company_profile(symbol: str, api_key: str, session: requests.Session) -> Dict:
    """Fetch company profile data from FMP API"""
    url = f"https://financialmodelingprep.com/api/v3/profile/{symbol}"
    params = {"apikey": api_key}
    
    try:
        response = session.get(url, params=params, timeout=10)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        logger.error(f"Error fetching {symbol}: {str(e)}")
        return None

def fetch_all_profiles(symbols: List[str], api_key: str) -> List[Dict]:
    session = create_session()
    
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [
            executor.submit(fetch_company_profile, symbol, api_key, session)
            for symbol in symbols
        ]
        results = [
            f.result() for f in concurrent.futures.as_completed(futures)
        ]
    
    return [r[0] for r in results if r and isinstance(r, list)]

# Execute fetching
company_profiles = fetch_all_profiles(symbols, FMP_API_KEY)

# Convert to Polars DataFrame more efficiently
if company_profiles:
    df = pl.DataFrame(company_profiles)

ERROR:__main__:Error fetching GOOG: HTTPSConnectionPool(host='financialmodelingprep.com', port=443): Max retries exceeded with url: /api/v3/profile/GOOG?apikey=PBkrv7HEcbYqq5BmXIRgFjfxzbLzuKmI (Caused by ResponseError('too many 429 error responses'))


ERROR:__main__:Error fetching META: HTTPSConnectionPool(host='financialmodelingprep.com', port=443): Max retries exceeded with url: /api/v3/profile/META?apikey=PBkrv7HEcbYqq5BmXIRgFjfxzbLzuKmI (Caused by ResponseError('too many 429 error responses'))


ERROR:__main__:Error fetching AAPL: HTTPSConnectionPool(host='financialmodelingprep.com', port=443): Max retries exceeded with url: /api/v3/profile/AAPL?apikey=PBkrv7HEcbYqq5BmXIRgFjfxzbLzuKmI (Caused by ResponseError('too many 429 error responses'))


ERROR:__main__:Error fetching NVDA: HTTPSConnectionPool(host='financialmodelingprep.com', port=443): Max retries exceeded with url: /api/v3/profile/NVDA?apikey=PBkrv7HEcbYqq5BmXIRgFjfxzbLzuKmI (Caused by ResponseError('too many 429 error responses'))


ERROR:__main__:Error fetching ORCL: HTTPSConnectionPool(host='financialmodelingprep.com', port=443): Max retries exceeded with url: /api/v3/profile/ORCL?apikey=PBkrv7HEcbYqq5BmXIRgFjfxzbLzuKmI (Caused by ResponseError('too many 429 error responses'))


...

ERROR:__main__:Error fetching APD: HTTPSConnectionPool(host='financialmodelingprep.com', port=443): Max retries exceeded with url: /api/v3/profile/APD?apikey=PBkrv7HEcbYqq5BmXIRgFjfxzbLzuKmI (Caused by ResponseError('too many 429 error responses'))


ERROR:__main__:Error fetching ECL: HTTPSConnectionPool(host='financialmodelingprep.com', port=443): Max retries exceeded with url: /api/v3/profile/ECL?apikey=PBkrv7HEcbYqq5BmXIRgFjfxzbLzuKmI (Caused by ResponseError('too many 429 error responses'))


ERROR:__main__:Error fetching PLD: HTTPSConnectionPool(host='financialmodelingprep.com', port=443): Max retries exceeded with url: /api/v3/profile/PLD?apikey=PBkrv7HEcbYqq5BmXIRgFjfxzbLzuKmI (Caused by ResponseError('too many 429 error responses'))


ERROR:__main__:Error fetching CCI: HTTPSConnectionPool(host='financialmodelingprep.com', port=443): Max retries exceeded with url: /api/v3/profile/CCI?apikey=PBkrv7HEcbYqq5BmXIRgFjfxzbLzuKmI (Caused by ResponseError('too many 429 error responses'))


ERROR:__main__:Error fetching AMT: HTTPSConnectionPool(host='financialmodelingprep.com', port=443): Max retries exceeded with url: /api/v3/profile/AMT?apikey=PBkrv7HEcbYqq5BmXIRgFjfxzbLzuKmI (Caused by ResponseError('too many 429 error responses'))


## Step 4: Clean Column Names

In [5]:
df = make_clean_names(df)

## Step 5: Write Polars to Parquet

In [6]:
output_dir = "../../../data/finance"

# Write DataFrame to Parquet
df.write_parquet(f'{output_dir}/company_profile.parquet')

## Step 6: Read Parquet (Validate)

In [7]:
pl.scan_parquet(f'{output_dir}/company_profile.parquet').head().collect()

symbol,price,beta,vol_avg,mkt_cap,last_div,range,changes,company_name,currency,cik,isin,cusip,exchange,exchange_short_name,industry,website,description,ceo,sector,country,full_time_employees,phone,address,city,state,zip,dcf_diff,dcf,image,ipo_date,default_image,is_etf,is_actively_trading,is_adr,is_fund
str,f64,f64,i64,i64,f64,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,null,null,f64,f64,str,str,bool,bool,bool,bool,bool
"""OSW""",19.59,2.13,515147,2037007380,0.08,"""11.98-20.57""",0.1,"""OneSpaWorld Holdings Limited""","""USD""","""0001758488""","""BSP736841136""","""P73684113""","""NASDAQ Capital Market""","""NASDAQ""","""Leisure""","""https://onespaworld.com""","""OneSpaWorld Holdings Limited o…","""Mr. Leonard I. Fluxman""","""Consumer Cyclical""","""BS""","""5024""","""3053589002""","""Office Number 2""","""Nassau""",null,null,35.80661,-16.216608,"""https://images.financialmodeli…","""2017-11-17""",false,false,true,false,false
